In [1]:
import fastNLP
import pandas as pd

from fastNLP.embeddings import BertEmbedding
from fastNLP.models import BertForQuestionAnswering
from fastNLP.core.losses import CMRC2018Loss
from fastNLP.core.metrics import CMRC2018Metric
from fastNLP.io.pipe.qa import CMRC2018BertPipe
from fastNLP import Trainer, BucketSampler
from fastNLP import WarmupCallback, GradientClipCallback
from fastNLP.core.optimizer import AdamW
from fastNLP import DataSet,Instance
from fastNLP.io import DataBundle
from copy import deepcopy
from fastNLP import DataSet, Instance

import numpy as np
# import pickle
import operator
# import pandas as pd
import jieba
# from language.langconv import *
import os

In [2]:
# qa_data=pd.read_csv('./data/qa/qa.csv')
translation_data=pd.read_csv('./data/translation/translation.csv')

In [3]:
# 由于是从那个问答系统的数据集改的

所以title，content保持和answers的值相同，answers_start我也设定成从0开始，这样应该就可以解决问题了，但是需要网上跑着试一试

In [3]:
translation_data.head()

,Unnamed: 0,input_texts,target_texts
0,0,Go.,Πάμε.
1,1,Run!,Τρέξε!
2,2,Run!,Τρέξτε!
3,3,Who?,Ποιος;
4,4,Wow!,Ουάου!


In [8]:
translation_data.shape

(10000, 3)

In [9]:
def get_DataBuddle(qa_data):
    question_texts=[str(j) for j in qa_data.input_texts]

    pre_answer_texts=[str(j) for j in qa_data.target_texts]

    answer_texts=[]

    for i in range(len(question_texts)):
        answer_texts.append([pre_answer_texts[i],pre_answer_texts[i],pre_answer_texts[i]])

    id_texts=[]
    for i in range(len(question_texts)):
        id_texts.append('DEV_0_QUERY_'+str(i))

    context_texts=pre_answer_texts
    title_texts=pre_answer_texts

    answer_starts_texts=[]
    for i in range(len(question_texts)):
        a=list()
        a.append(0)
        a.append(0)
        a.append(0)
        answer_starts_texts.append(a)
    ds = DataSet()

    for i in range(len(question_texts)):
        title=title_texts[i]
        context=context_texts[i]
        question=question_texts[i]
        answers=answer_texts[i]
        answer_starts=answer_starts_texts
        id=id_texts
        ds.append(Instance(title=title, context=context, question=question, answers=answers,
                                           answer_starts=answer_starts,id=id))
    return ds

In [10]:
ds=get_DataBuddle(translation_data)

In [11]:
data_bundle=DataBundle()
train_data, dev_data = ds.split(0.2, shuffle=False)
data_bundle.set_dataset(train_data, 'train')
data_bundle.set_dataset(dev_data, 'dev')

In total 2 datasets:
	train has 8000 instances.
	dev has 2000 instances.

# 预处理

获取分词器，对于不同语种就换一种分词器就行，这个以待研究

In [13]:
max_len=510

In [15]:
tokenize_method='spacy'
# tokenize_method='spacy'
lang='en_core_web_sm'
def _raw_split(sent):
    return sent.split()
def _cn_char_split(sent):
    return [chars for chars in sent]
r"""

:param str tokenize_method: 获取tokenzier方法
:param str lang: 语言，当前仅支持en
:return: 返回tokenize函数
"""
tokenizer_dict = {
    'spacy': None,
    'raw': _raw_split,
    'cn-char': _cn_char_split,
}
if tokenize_method == 'spacy':
    import spacy
    spacy.prefer_gpu()
    if lang != 'en_core_web_sm':
        raise RuntimeError("Spacy only supports en right right.")
    # 估计目前是不支持spacy用法，所以才有这么回事
    #到时候想法子改一下参数就行了
    en = spacy.load(lang)
    tokenizer = lambda x: [w.text for w in en.tokenizer(x)]
elif tokenize_method in tokenizer_dict:
    tokenizer = tokenizer_dict[tokenize_method]
else:
    raise RuntimeError(f"Only support {tokenizer_dict.keys()} tokenizer.")

In [16]:
def _concat_clip(data_bundle, max_len, concat_field_name='raw_chars'):
    r"""
    处理data_bundle中的DataSet，将context与question按照character进行tokenize，然后使用[SEP]将两者连接起来。

    会新增field: context_len(int), raw_words(list[str]), target_start(int), target_end(int)其中target_start
    与target_end是与raw_chars等长的。其中target_start和target_end是前闭后闭的区间。

    :param DataBundle data_bundle: 类似["a", "b", "[SEP]", "c", ]
    :return:
    """
#     tokenizer = get_tokenizer('cn-char', lang='cn')
    for name in list(data_bundle.datasets.keys()):
        ds = data_bundle.get_dataset(name)
        data_bundle.delete_dataset(name)
        new_ds = DataSet()
        # i=0
        for ins in ds:
        #     i=i+1
            new_ins = deepcopy(ins)
            context = ins['context']
            question = ins['question']
        #     print(context)
        #     print(question)
            cnt_lst = tokenizer(context)
        #     print(cnt_lst)
            q_lst = tokenizer(question)
        #     print(q_lst)
            answer_start = -1
        #     print('=='*20)
            print(len(cnt_lst))
            print(len(q_lst))
            print(len(cnt_lst) + len(q_lst) + 3)
            if len(cnt_lst) + len(q_lst) + 3 > max_len:  
                # 预留开头的[CLS]和[SEP]和中间的[sep]
                if 'answer_starts' in ins and 'answers' in ins:
                    answer_start = int(ins['answer_starts'][0][0])
                    print('answer_start',answer_start)
                    answer = ins['answers'][0]
                    answer_end = answer_start + len(answer)
                    if answer_end > max_len - 3 - len(q_lst):
                        span_start = answer_end + 3 + len(q_lst) - max_len
                        span_end = answer_end
                    else:
                        span_start = 0
                        span_end = max_len - 3 - len(q_lst)
                    cnt_lst = cnt_lst[span_start:span_end]
                    answer_start = int(ins['answer_starts'][0][0])
                    answer_start -= span_start
                    answer_end = answer_start + len(ins['answers'][0])
                else:
                    cnt_lst = cnt_lst[:max_len - len(q_lst) - 3]
            else:
                if 'answer_starts' in ins and 'answers' in ins:
                    print('='*20)
                    print(ins['answer_starts'][0])
                    answer_start = int(ins['answer_starts'][0][0])
                    print('<'*20)
                    print(answer_start)
                    answer_end = answer_start + len(ins['answers'][0])
                    print(ins['answers'])
                    print(answer_end)
            tokens = cnt_lst + ['[SEP]'] + q_lst
            new_ins['context_len'] = len(cnt_lst)
            new_ins['raw_chars'] = tokens
            if answer_start != -1:
                    new_ins['target_start'] = answer_start
        #             print(answer_start)
                    new_ins['target_end'] = answer_end - 1
        #             print(answer_end-1)
            new_ds.append(new_ins)
            data_bundle.set_dataset(new_ds, name)

    return data_bundle

In [ ]:
data_bundle = _concat_clip(data_bundle
                                   , max_len=max_len
                                   , concat_field_name='raw_chars')

3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Χρειάζεται χρήματα.', 'Χρειάζεται χρήματα.', 'Χρειάζεται χρήματα.']
19
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Με σέβεται.', 'Με σέβεται.', 'Με σέβεται.']
11
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Φαίνεται χαρούμενος.', 'Φαίνεται χαρούμενος.', 'Φαίνεται χαρούμενος.']
20
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Φαίνεται ευτυχισμένος.', 'Φαίνεται ευτυχισμένος.', 'Φαίνεται ευτυχισμένος.']
22
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Δείχνει κουρασμένος.', 'Δείχνει κουρασμένος.', 'Δείχνει κουρασμένος.']
20
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Φαίνεται κουρασμένος.', 'Φαίνεται κουρασμένος.', 'Φαίνεται κουρασμένος.']
21
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Σε εμπιστεύτηκε.', 'Σε εμπιστεύτηκε.', 'Σε εμπιστεύτηκε.']
16
4
4
11
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
["Σ' εμπιστεύτηκε.", "Σ' εμπιστεύτηκε.", "Σ' εμπιστεύτηκε."]
16
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Σας εμπιστεύτηκε.', 'Σας εμπιστεύτηκε.', 'Σας εμπιστεύτηκε.']
17
3
4
10
[0,

[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Δεν πλήρωσα τίποτα.', 'Δεν πλήρωσα τίποτα.', 'Δεν πλήρωσα τίποτα.']
19
4
4
11
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Δεν είπα τίποτα.', 'Δεν είπα τίποτα.', 'Δεν είπα τίποτα.']
16
4
5
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είδα πέντε άντρες.', 'Είδα πέντε άντρες.', 'Είδα πέντε άντρες.']
18
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Πουλάω λουλούδια.', 'Πουλάω λουλούδια.', 'Πουλάω λουλούδια.']
17
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Μιλάω Γαλλικά.', 'Μιλάω Γαλλικά.', 'Μιλάω Γαλλικά.']
14
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Διδάσκω Γαλλικά.', 'Διδάσκω Γαλλικά.', 'Διδάσκω Γαλλικά.']
16
4
5
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Νομίζω ότι μπορούμε.', 'Νομίζω ότι μπορούμε.', 'Νομίζω ότι μπορούμε.']
20
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Έστριψα δεξιά.', 'Έστριψα δεξιά.', 'Έστριψα δεξιά.']
14
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Συνήθως περπατάω.', 'Συνήθως περπατάω.', 'Συνήθως περπατάω.']
17
4
5
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Θέλω ένα

16
4
5
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Θα είναι έτοιμη.', 'Θα είναι έτοιμη.', 'Θα είναι έτοιμη.']
16
5
5
13
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
["Θα 'ναι έτοιμος.", "Θα 'ναι έτοιμος.", "Θα 'ναι έτοιμος."]
16
5
5
13
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
["Θα 'ναι έτοιμη.", "Θα 'ναι έτοιμη.", "Θα 'ναι έτοιμη."]
15
5
5
13
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
["Θα 'ναι έτοιμο.", "Θα 'ναι έτοιμο.", "Θα 'ναι έτοιμο."]
15
3
5
11
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είναι ολοκαίνουργιο.', 'Είναι ολοκαίνουργιο.', 'Είναι ολοκαίνουργιο.']
20
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είναι επικίνδυνο!', 'Είναι επικίνδυνο!', 'Είναι επικίνδυνο!']
17
4
4
11
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
["Είν' επικίνδυνο!", "Είν' επικίνδυνο!", "Είν' επικίνδυνο!"]
16
3
4
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είναι λάθος.', 'Είναι λάθος.', 'Είναι λάθος.']
12
4
5
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Δεν είναι φτηνός.', 'Δεν είναι φτηνός.', 'Δεν είναι φτηνός.']
17
4
5
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Δεν είναι φτηνή.', 

7
5
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ο Τομ δεν μπορεί να βαδίσει.', 'Ο Τομ δεν μπορεί να βαδίσει.', 'Ο Τομ δεν μπορεί να βαδίσει.']
28
7
5
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Δεν μπορεί να περπατήσει ο Τομ.', 'Δεν μπορεί να περπατήσει ο Τομ.', 'Δεν μπορεί να περπατήσει ο Τομ.']
31
7
5
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Δεν μπορεί να βαδίσει ο Τομ.', 'Δεν μπορεί να βαδίσει ο Τομ.', 'Δεν μπορεί να βαδίσει ο Τομ.']
28
4
3
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ο Τομ παραπονέθηκε.', 'Ο Τομ παραπονέθηκε.', 'Ο Τομ παραπονέθηκε.']
19
5
4
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ο Τομ μαγειρεύει καλά.', 'Ο Τομ μαγειρεύει καλά.', 'Ο Τομ μαγειρεύει καλά.']
22
5
4
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ο Τομ πέθανε νέος.', 'Ο Τομ πέθανε νέος.', 'Ο Τομ πέθανε νέος.']
18
5
5
13
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ο Τομ τρώει πολύ.', 'Ο Τομ τρώει πολύ.', 'Ο Τομ τρώει πολύ.']
17
5
5
13
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Τρώει πολύ ο Τομ.', 'Τρώει πολύ ο Τομ.', 'Τρώει πολύ ο Τομ.']
17
5
4
12
[0, 0

[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ο Τομ κράτησε σημειώσεις.', 'Ο Τομ κράτησε σημειώσεις.', 'Ο Τομ κράτησε σημειώσεις.']
25
6
4
13
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
["Ο Τομ σ' εμπιστεύεται.", "Ο Τομ σ' εμπιστεύεται.", "Ο Τομ σ' εμπιστεύεται."]
22
5
4
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ο Τομ σε εμπιστεύεται.', 'Ο Τομ σε εμπιστεύεται.', 'Ο Τομ σε εμπιστεύεται.']
22
5
4
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ο Τομ σας εμπιστεύεται.', 'Ο Τομ σας εμπιστεύεται.', 'Ο Τομ σας εμπιστεύεται.']
23
4
3
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ο Τομ κατάλαβε.', 'Ο Τομ κατάλαβε.', 'Ο Τομ κατάλαβε.']
15
6
4
13
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ο Τομ θέλει τη Μαίρη.', 'Ο Τομ θέλει τη Μαίρη.', 'Ο Τομ θέλει τη Μαίρη.']
21
5
4
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ο Τομ ήταν διάσημος.', 'Ο Τομ ήταν διάσημος.', 'Ο Τομ ήταν διάσημος.']
20
4
4
11
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ο Τομ πεινούσε.', 'Ο Τομ πεινούσε.', 'Ο Τομ πεινούσε.']
15
4
4
11
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Πεινούσε ο Τομ.', 'Πεινούσε 

...

27
6
6
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Μην ανοίξετε αυτό το κουτί.', 'Μην ανοίξετε αυτό το κουτί.', 'Μην ανοίξετε αυτό το κουτί.']
27
5
6
14
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Μη στέκεσαι κοντά μου.', 'Μη στέκεσαι κοντά μου.', 'Μη στέκεσαι κοντά μου.']
22
6
4
13
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Όλος ο κόσμος τον αγαπάει.', 'Όλος ο κόσμος τον αγαπάει.', 'Όλος ο κόσμος τον αγαπάει.']
26
4
4
11
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Καθένας είναι ξεχωριστός.', 'Καθένας είναι ξεχωριστός.', 'Καθένας είναι ξεχωριστός.']
25
4
4
11
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Όλα είναι έτοιμα.', 'Όλα είναι έτοιμα.', 'Όλα είναι έτοιμα.']
17
4
7
14
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Συγνώμη, χάθηκα.', 'Συγνώμη, χάθηκα.', 'Συγνώμη, χάθηκα.']
16
5
4
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Τα Γαλλικά είναι δύσκολα.', 'Τα Γαλλικά είναι δύσκολα.', 'Τα Γαλλικά είναι δύσκολα.']
25
5
4
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είναι δύσκολα τα Γαλλικά.', 'Είναι δύσκολα τα Γαλλικά.', 'Είναι δύσκολα τα Γαλλικά.']
25


5
7
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Δε θέλω να φάω.', 'Δε θέλω να φάω.', 'Δε θέλω να φάω.']
15
5
5
13
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Βλέπω όνειρα κάθε βράδυ.', 'Βλέπω όνειρα κάθε βράδυ.', 'Βλέπω όνειρα κάθε βράδυ.']
24
5
6
14
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Οδηγώ ένα μαύρο αμάξι.', 'Οδηγώ ένα μαύρο αμάξι.', 'Οδηγώ ένα μαύρο αμάξι.']
22
5
6
14
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Οδηγάω ένα μαύρο αυτοκίνητο.', 'Οδηγάω ένα μαύρο αυτοκίνητο.', 'Οδηγάω ένα μαύρο αυτοκίνητο.']
28
4
5
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ένιωσα πολύ άβολα.', 'Ένιωσα πολύ άβολα.', 'Ένιωσα πολύ άβολα.']
18
4
5
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είχα καλούς δασκάλους.', 'Είχα καλούς δασκάλους.', 'Είχα καλούς δασκάλους.']
22
4
5
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είχα καλές δασκάλες.', 'Είχα καλές δασκάλες.', 'Είχα καλές δασκάλες.']
20
4
5
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είχα τρία εισιτήρια.', 'Είχα τρία εισιτήρια.', 'Είχα τρία εισιτήρια.']
20
6
7
16
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Πρέ

0
['Δεν παραπονιέμαι.', 'Δεν παραπονιέμαι.', 'Δεν παραπονιέμαι.']
17
7
6
16
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
["Δεν είμαι απ' τη Βοστώνη.", "Δεν είμαι απ' τη Βοστώνη.", "Δεν είμαι απ' τη Βοστώνη."]
25
6
6
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Δεν είμαι από τη Βοστώνη.', 'Δεν είμαι από τη Βοστώνη.', 'Δεν είμαι από τη Βοστώνη.']
25
4
6
13
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Δεν πεινάω πολύ.', 'Δεν πεινάω πολύ.', 'Δεν πεινάω πολύ.']
16
5
6
14
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Δεν πεινάω και πολύ.', 'Δεν πεινάω και πολύ.', 'Δεν πεινάω και πολύ.']
20
7
7
17
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είμαι περήφανος για τον γιο μου.', 'Είμαι περήφανος για τον γιο μου.', 'Είμαι περήφανος για τον γιο μου.']
32
7
7
17
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είμαι περήφανη για τον γιο μου.', 'Είμαι περήφανη για τον γιο μου.', 'Είμαι περήφανη για τον γιο μου.']
31
3
5
11
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Μελετάω Γαλλικά.', 'Μελετάω Γαλλικά.', 'Μελετάω Γαλλικά.']
16
3
5
11
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Δι

4
5
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είναι πολύ επικίνδυνο.', 'Είναι πολύ επικίνδυνο.', 'Είναι πολύ επικίνδυνο.']
22
6
6
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είναι η μόνη σας ελπίδα.', 'Είναι η μόνη σας ελπίδα.', 'Είναι η μόνη σας ελπίδα.']
24
6
6
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Είναι η μόνη σου ελπίδα.', 'Είναι η μόνη σου ελπίδα.', 'Είναι η μόνη σου ελπίδα.']
24
5
4
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Οι Γιαπωνέζοι είναι Ασιάτες.', 'Οι Γιαπωνέζοι είναι Ασιάτες.', 'Οι Γιαπωνέζοι είναι Ασιάτες.']
28
5
4
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Οι Ιάπωνες είναι Ασιάτες.', 'Οι Ιάπωνες είναι Ασιάτες.', 'Οι Ιάπωνες είναι Ασιάτες.']
25
6
6
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ας ελπίσουμε πως θα πετύχει.', 'Ας ελπίσουμε πως θα πετύχει.', 'Ας ελπίσουμε πως θα πετύχει.']
28
5
6
14
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ας το κάνουμε συνοπτικά.', 'Ας το κάνουμε συνοπτικά.', 'Ας το κάνουμε συνοπτικά.']
24
5
6
14
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Ας παραγγείλουμε μια πίτσα.', 'Ας παραγγε

['Πες μου ποιον είδες.', 'Πες μου ποιον είδες.', 'Πες μου ποιον είδες.']
20
6
6
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Πες μας πού να πάμε.', 'Πες μας πού να πάμε.', 'Πες μας πού να πάμε.']
20
6
6
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Πείτε μας πού να πάμε.', 'Πείτε μας πού να πάμε.', 'Πείτε μας πού να πάμε.']
22
6
5
14
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Η Ταϊλάνδη βρίσκεται στην Ασία.', 'Η Ταϊλάνδη βρίσκεται στην Ασία.', 'Η Ταϊλάνδη βρίσκεται στην Ασία.']
31
6
5
14
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Εκείνο το φόρεμα είναι φτηνό.', 'Εκείνο το φόρεμα είναι φτηνό.', 'Εκείνο το φόρεμα είναι φτηνό.']
29
7
5
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Εκείνος ο μαθητής είναι ο Τομ.', 'Εκείνος ο μαθητής είναι ο Τομ.', 'Εκείνος ο μαθητής είναι ο Τομ.']
30
7
5
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Εκείνος ο φοιτητής είναι ο Τομ.', 'Εκείνος ο φοιτητής είναι ο Τομ.', 'Εκείνος ο φοιτητής είναι ο Τομ.']
31
5
6
14
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['Αυτό ήταν καλό καπέλο.', 'Αυτό ήταν καλό καπέλο.', 'Αυτό 

In [ ]:
data_bundle.get_dataset('dev')

# 预处理

In [20]:
from fastNLP import Vocabulary

In [25]:
def process(data_bundle: DataBundle) -> DataBundle:
    data_bundle = _concat_clip(data_bundle
                                   , max_len=max_len
                                   , concat_field_name='raw_chars')

    src_vocab = Vocabulary()
    src_vocab.from_dataset(*[ds for name, ds in data_bundle.iter_datasets() 
                             if 'train' in name],
                           field_name='raw_chars',
                           no_create_entry_dataset=[ds for name, ds in data_bundle.iter_datasets()
                                                    if 'train' not in name]
                           )
    src_vocab.index_dataset(*data_bundle.datasets.values()
                            , field_name='raw_chars', new_field_name='chars')
    data_bundle.set_vocab(src_vocab, 'chars')

    data_bundle.set_ignore_type('raw_chars', 'answers', flag=True)
    data_bundle.set_input('chars')
    data_bundle.set_target('raw_chars', 
                           'answers',
                           'target_start',
                           'target_end',
                           'context_len')

    return data_bundle

In [26]:
data_bundle=process(data_bundle)

12
6
21
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['我试着尽可能好,我可以。', '我试着尽可能好,我可以。', '我试着尽可能好,我可以。']
12
14
5
22
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['幸福不是真正的可预测的情绪。', '幸福不是真正的可预测的情绪。', '幸福不是真正的可预测的情绪。']
14
13
5
21
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['耻辱是一种常见的人类情绪。', '耻辱是一种常见的人类情绪。', '耻辱是一种常见的人类情绪。']
13
6
6
15
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['你能感觉到吗', '你能感觉到吗', '你能感觉到吗']
6
4
5
12
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['你感觉。', '你感觉。', '你感觉。']
4
3
3
9
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['你感觉', '你感觉', '你感觉']
3
4
3
10
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['你觉得爱', '你觉得爱', '你觉得爱']
4
12
4
19
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['我们都对自己的感情负责。', '我们都对自己的感情负责。', '我们都对自己的感情负责。']
12
22
6
31
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['我没有任何情绪,所以我不能真正感到悲伤这样。', '我没有任何情绪,所以我不能真正感到悲伤这样。', '我没有任何情绪,所以我不能真正感到悲伤这样。']
22
3
2
8
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['你感觉', '你感觉', '你感觉']
3
20
7
30
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['恐惧是一种人类的情感,机器人不经历恐惧。', '恐惧是一种人类的情感,机器人不经历恐惧。', '恐惧是一种人类的情感,机器人不经历恐惧。']
20
8
7
18
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['我没有什

13
12
9
24
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['全球性组织推动环保行动。', '全球性组织推动环保行动。', '全球性组织推动环保行动。']
12
27
5
35
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['政治管理建立制度,以此来一个民族,国家,地区等的管理。', '政治管理建立制度,以此来一个民族,国家,地区等的管理。', '政治管理建立制度,以此来一个民族,国家,地区等的管理。']
27
39
7
49
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['一个社会政治运动主张生产资料,并带来一个无阶级社会的阶级解决冲突的共同所有权。', '一个社会政治运动主张生产资料,并带来一个无阶级社会的阶级解决冲突的共同所有权。', '一个社会政治运动主张生产资料,并带来一个无阶级社会的阶级解决冲突的共同所有权。']
39
18
5
26
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['当一个人的名誉或声誉受到质疑或怀疑。', '当一个人的名誉或声誉受到质疑或怀疑。', '当一个人的名誉或声誉受到质疑或怀疑。']
18
10
5
18
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['这是完全可以理解的。', '这是完全可以理解的。', '这是完全可以理解的。']
10
10
5
18
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['怎么样,第二修正案？', '怎么样,第二修正案？', '怎么样,第二修正案？']
10
13
4
20
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['不是特别。我不是暴力行为。', '不是特别。我不是暴力行为。', '不是特别。我不是暴力行为。']
13
10
4
17
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['我支持了第二次修正。', '我支持了第二次修正。', '我支持了第二次修正。']
10
8
9
20
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['安德鲁·杰克逊。', '安德鲁·杰克逊。', '安德鲁·杰克逊。']
8
10
4
17
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['它每

[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['金手套。', '金手套。', '金手套。']
4
4
17
24
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['滑雪板。', '滑雪板。', '滑雪板。']
4
9
5
17
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['一个游戏高大队员。', '一个游戏高大队员。', '一个游戏高大队员。']
9
10
4
17
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['我天生没有运动基因。', '我天生没有运动基因。', '我天生没有运动基因。']
10
54
5
62
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['一个游戏用硬,生皮覆盖球和木棒打了由每九或十的选手两个对立的球队。它是在一个领域发挥与四种碱基形成菱形电路。', '一个游戏用硬,生皮覆盖球和木棒打了由每九或十的选手两个对立的球队。它是在一个领域发挥与四种碱基形成菱形电路。', '一个游戏用硬,生皮覆盖球和木棒打了由每九或十的选手两个对立的球队。它是在一个领域发挥与四种碱基形成菱形电路。']
54
57
5
65
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['一个游戏一个圆球扮演的11名队员组成,两队在现场与在任一端目标;球被踢或用身体除了手和手臂的任何部分,主要是感动。', '一个游戏一个圆球扮演的11名队员组成,两队在现场与在任一端目标;球被踢或用身体除了手和手臂的任何部分,主要是感动。', '一个游戏一个圆球扮演的11名队员组成,两队在现场与在任一端目标;球被踢或用身体除了手和手臂的任何部分,主要是感动。']
57
10
4
17
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['我不喜欢运动那么多。', '我不喜欢运动那么多。', '我不喜欢运动那么多。']
10
22
4
29
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['你必须在运行时运行速度非常快,以得到什么好处', '你必须在运行时运行速度非常快,以得到什么好处', '你必须在运行时运行速度非常快,以得到什么好处']
22
9
4
16
[0, 0, 0]
<<<<<<<<<<<<<<<<<<<<
0
['你喜欢哪

In [28]:
print(data_bundle)

In total 2 datasets:
	train has 344 instances.
	dev has 86 instances.
In total 1 vocabs:
	chars has 1123 entries.



In [29]:
data_bundle.rename_field('chars', 'words')

In total 2 datasets:
	train has 344 instances.
	dev has 86 instances.
In total 1 vocabs:
	words has 1123 entries.

In [30]:
data_bundle.get_dataset('dev')

+------------+--------------+---------------+--------------+-------------------+-----------+-------------+---------------+--------------+------------+-----------+
| title      | context      | question      | answers      | answer_starts     | id        | context_len | raw_chars     | target_start | target_end | words     |
+------------+--------------+---------------+--------------+-------------------+-----------+-------------+---------------+--------------+------------+-----------+
| 人工智...  | 人工智能...  | 什么是ai      | ['人工智...  | [[0, 0, 0], [0... | ['DEV_... | 25          | ['人', '工... | 0            | 24         | [14, 9... |
| 蟒蛇       | 蟒蛇         | 你写的是什... | ['蟒蛇',...  | [[0, 0, 0], [0... | ['DEV_... | 2           | ['蟒', '蛇... | 0            | 1          | [1063,... |
| 是的,我... | 是的,我受... | 你听起来像... | ['是的,我... | [[0, 0, 0], [0... | ['DEV_... | 19          | ['是', '的... | 0            | 18         | [3, 6,... |
| 那是我...  | 那是我的...  | 你是一个人... | ['那是我...  | [[0, 0, 0], [0... | [

In [27]:
data_bundle.get_dataset('dev')

+------------+--------------+---------------+--------------+-------------------+-----------+-------------+---------------+--------------+------------+-----------+
| title      | context      | question      | answers      | answer_starts     | id        | context_len | raw_chars     | target_start | target_end | chars     |
+------------+--------------+---------------+--------------+-------------------+-----------+-------------+---------------+--------------+------------+-----------+
| 人工智...  | 人工智能...  | 什么是ai      | ['人工智...  | [[0, 0, 0], [0... | ['DEV_... | 25          | ['人', '工... | 0            | 24         | [14, 9... |
| 蟒蛇       | 蟒蛇         | 你写的是什... | ['蟒蛇',...  | [[0, 0, 0], [0... | ['DEV_... | 2           | ['蟒', '蛇... | 0            | 1          | [1063,... |
| 是的,我... | 是的,我受... | 你听起来像... | ['是的,我... | [[0, 0, 0], [0... | ['DEV_... | 19          | ['是', '的... | 0            | 18         | [3, 6,... |
| 那是我...  | 那是我的...  | 你是一个人... | ['那是我...  | [[0, 0, 0], [0... | [

In [198]:
data_bundle.rename_field('chars', 'words')

In total 2 datasets:
	train has 344 instances.
	dev has 86 instances.
In total 1 vocabs:
	words has 1123 entries.

In [199]:
data_bundle.get_dataset('dev')

+------------+--------------+---------------+--------------+-------------------+-----------+-------------+---------------+--------------+------------+-----------+
| title      | context      | question      | answers      | answer_starts     | id        | context_len | raw_chars     | target_start | target_end | words     |
+------------+--------------+---------------+--------------+-------------------+-----------+-------------+---------------+--------------+------------+-----------+
| 人工智...  | 人工智能...  | 什么是ai      | ['人工智...  | [[0, 0, 0], [0... | ['DEV_... | 25          | ['人', '工... | 0            | 24         | [14, 9... |
| 蟒蛇       | 蟒蛇         | 你写的是什... | ['蟒蛇',...  | [[0, 0, 0], [0... | ['DEV_... | 2           | ['蟒', '蛇... | 0            | 1          | [1063,... |
| 是的,我... | 是的,我受... | 你听起来像... | ['是的,我... | [[0, 0, 0], [0... | ['DEV_... | 19          | ['是', '的... | 0            | 18         | [3, 6,... |
| 那是我...  | 那是我的...  | 你是一个人... | ['那是我...  | [[0, 0, 0], [0... | [

In [200]:
data_bundle.datasets.keys()

dict_keys(['train', 'dev'])

In [201]:
ds = data_bundle.get_dataset('dev')

# 这样应该组建的数据集已经弄成功了

In [ ]:
embed = BertEmbedding(data_bundle.get_vocab('words'), model_dir_or_name='cn', requires_grad=True, include_cls_sep=False, auto_truncate=True,
                      dropout=0.5, word_dropout=0.01)
model = BertForQuestionAnswering(embed)
loss = CMRC2018Loss()
# metric = CMRC2018Metric()

  0%|                                                                               | 98.3k/412M [00:00<07:10, 955kB/s]

http://212.129.155.247/embedding/bert-chinese-wwm.zip not found in cache, downloading to C:\Users\ADMINI~1\AppData\Local\Temp\tmpod_4zw9w


  3%|██▍                                                                           | 12.8M/412M [00:03<01:40, 3.98MB/s]

In [ ]:


wm_callback = WarmupCallback(schedule='linear')
gc_callback = GradientClipCallback(clip_value=1, clip_type='norm')
callbacks = [wm_callback, gc_callback]

optimizer = AdamW(model.parameters(), lr=5e-5)

trainer = Trainer(data_bundle.get_dataset('train')
                  , model, loss=loss, optimizer=optimizer,
                  sampler=BucketSampler(seq_len_field_name='context_len'),
                  dev_data=data_bundle.get_dataset('dev')
                  , metrics=metric,
                  callbacks=callbacks,
                  batch_size=6, num_workers=2, n_epochs=2, print_every=1,
                  test_use_tqdm=False, update_every=10)

trainer.train(load_best_model=False)